### ConversationKGMemory

지식 그래프를 활용하여 메모리 저장.<br>
모델이 서로 다른 개체 간의 관계를 이해하는데 도움을 주고, 복잡한 연결망과 역사적 맥락을 기반으로 대응하는 능력 향상

In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging

logging.langsmith("ConversationKGMemory")

LangSmith 추적을 시작합니다.
[프로젝트명]
ConversationKGMemory


In [2]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

In [3]:
llm = ChatOpenAI(model_name = 'gpt-4o-mini', temperature=0)

memory = ConversationKGMemory(llm = llm , return_messages=True)
memory.save_context(
    {"input": "이쪽은 Pangyo 에 거주중인 김셜리씨 입니다."},
    {"output": "김셜리씨는 누구시죠?"},
)
memory.save_context(
    {"input": "김셜리씨는 우리 회사의 신입 디자이너입니다."},
    {"output": "만나서 반갑습니다."},
)

In [4]:
memory.load_memory_variables({"input": "김셜리씨는 누구입니까?"})

{'history': [SystemMessage(content='On 김셜리씨: 김셜리씨 is 신입 디자이너. 김셜리씨 is in 우리 회사.')]}

### Chain에 메모리 활용

In [6]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(model_name = 'gpt-4o-mini', temperature=0)

template = """
The following is a freindly conversation between a human and AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to qeustion, it truthfully says it does not know.
The AI ONLY uees information contained in the "Relevant Information" secion and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""

prompt = PromptTemplate(
    input_variables = ['history', 'input'],
    template = template
    )
conversation_with_kg = ConversationChain(
    llm = llm,
    prompt = prompt,
    memory = ConversationKGMemory(llm = llm)
)


In [8]:
conversation_with_kg.predict(
    input = "My name is Teddy. Shirlry is a coworker of mine, and she's new designer at our company."
)

"Hello, Teddy! It's great to meet you. How's it going with Shirley, the new designer? Have you had a chance to work together on any projects yet?"

In [9]:
conversation_with_kg.memory.load_memory_variables({"input": "Who is Shirley?"})

{'history': 'On Shirley: Shirley is a coworker. Shirley is a new designer. Shirley works at the company.'}